# GenreMatch
Data cleaning and model generation  
*Jeremy Freedman, Reza Madhavan, Kunal Sheth*

In [2]:
!unzip Archive.zip

Archive:  Archive.zip
replace utils/genius.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace utils/grab_lyrics.py? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
!ls utils

azlyrics.py  grab_genius.ipynb	__pycache__
genius.py    grab_lyrics.py	songlyrics.py


In [2]:
import pandas as pd
import numpy as np
import songlyrics as sl
from collections import defaultdict
from nltk.corpus import stopwords as sw
from nltk import download as nltk_download
import sklearn as sk
from sklearn import tree
import collections
import gc

In [3]:
a=pd.read_csv('data/song_lyrics.csv').dropna()
new_df = a.copy()
head = new_df.head(5)

In [4]:
# download the NLTK stopwords list, if necessary
nltk_download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
# we need to clean and update the NLTK stopwords for our data
# we're stripping out punctuation entirely, which the stopwords are not equipped to handle
# could optionally add music-centric stopwords ('oh', 'yeah', 'like', etc)

temp_words = sw.words('english')
stopwords = []
additions = ['im', 'ill', 'id', 'oh', 'cant', 'ive']
for w in temp_words:
    stopwords.append(sl._clean(w))
stopwords += additions
print(stopwords)


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'youre', 'youve', 'youll', 'youd', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'shes', 'her', 'hers', 'herself', 'it', 'its', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'thatll', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', '

In [7]:
for index,row in new_df.iterrows():
  if index % 100 == 0:
    print(index)

  song = row['Lyrics']
  x=song.find('\n')
  song2 = sl._clean(song[x+1:])
  new_song = ''
  for i in range(len(song2)):
    char = song2[i]

    if char == '\n':
      new_song += ' '
    elif char == '(' or char == ')':
      new_song += ''
    else:
      new_song += char


    new_song = new_song.encode('ascii','ignore').decode()

  song2 = new_song.split(' ')
  song2 = list(filter(lambda x : x!='' and x not in stopwords, song2))
  song2 = song2[:len(song2)-1]

  new_df.at[index, 'Lyrics'] = song2


0


AttributeError: ignored

In [8]:
wordfreq = []
for index,row in new_df.iterrows():
  lyrics = row['Lyrics']

  x = dict(collections.Counter(lyrics))
  # mapped = []
  # for k in x.keys():
  #   mapped.append((k,x[k]))
  
  wordfreq.append(x)

new_df['WordFreq'] = wordfreq

In [9]:
new_df['UniqueWords']= new_df.apply(lambda row : len(row['WordFreq']), axis = 1)
new_df

,Unnamed: 0,Artist,Song,Lyrics,Genre,WordFreq,UniqueWords
0,0,One Direction,Perfect,"[might, never, knight, shinin, armor, might, n...",pop,"{'might': 4, 'never': 4, 'knight': 1, 'shinin'...",71
1,1,One Direction,Story of My Life,"[written, walls, stories, explain, leave, hear...",pop,"{'written': 3, 'walls': 2, 'stories': 1, 'expl...",68
2,2,One Direction,Drag Me Down,"[got, fire, heart, scared, dark, never, seen, ...",pop,"{'got': 4, 'fire': 2, 'heart': 2, 'scared': 2,...",37
3,3,One Direction,What Makes You Beautiful,"[insecure, know, turning, heads, walk, door, n...",pop,"{'insecure': 1, 'know': 18, 'turning': 1, 'hea...",54
4,4,One Direction,Night Changes,"[going, tonight, changes, something, red, moth...",pop,"{'going': 2, 'tonight': 3, 'changes': 8, 'some...",65
...,...,...,...,...,...,...,...
4074,4074,Def Leppard,Action! Not Words,"[sick, tired, damn, tv, gonna, make, movie, wa...",metal,"{'sick': 1, 'tired': 1, 'damn': 1, 'tv': 1, 'g...",61
4075,4075,Def Leppard,Rock Rock (Till You Drop),"[ready, get, set, tear, place, apart, need, ti...",metal,"{'ready': 1, 'get': 2, 'set': 1, 'tear': 1, 'p...",74
4076,4076,Def Leppard,Slang,"[sitting, dark, getting, taken, said, somethin...",metal,"{'sitting': 1, 'dark': 1, 'getting': 1, 'taken...",41
4077,4077,Def Leppard,Die Hard the Hunter,"[lets, welcome, home, soldier, boy, far, away,...",metal,"{'lets': 2, 'welcome': 2, 'home': 2, 'soldier'...",65


In [10]:
uniq_words = set()
for x in new_df['WordFreq']:
  uniq_words = uniq_words.union(x.keys())
uniq_wordict = {k : 0 for k in uniq_words}

In [11]:
word_df = new_df[['Genre', 'WordFreq', 'Artist', 'UniqueWords']]

for index,row in word_df.iterrows():
  for k in row['WordFreq']:
    uniq_wordict[k] = row['WordFreq'][k]

  word_df.at[index, 'WordFreq'] = uniq_wordict
  word_df.at[index, 'Artist'] = list(uniq_wordict.values()) + [row['UniqueWords']]
  uniq_wordict = {k : 0 for k in uniq_words}

word_df = word_df.replace({'Artist':'WordFreqList'})


In [12]:
gc.collect()

142

In [13]:
word_df = word_df.rename(columns = {'Artist':'WordFreqList'})

In [14]:
# make a 80/20 train/test split of lyric words tagged by origin genre
X_train, X_test, Y_train, Y_test = sk.model_selection.train_test_split(list(word_df['WordFreqList']), list(word_df['Genre']), test_size=0.2)

print(f'Split size: {len(X_test) / (len(X_test) + len(X_train))}\nTest set: {len(X_test)}\nTrain set: {len(X_train)}')

Split size: 0.20009808729769496
Test set: 816
Train set: 3262


In [20]:
X_train[1][-1]

66

In [15]:
clf = tree.DecisionTreeClassifier()
clf.fit(X_train,Y_train)

DecisionTreeClassifier()

In [16]:
preds = clf.predict(X_test)

In [17]:
np.sum(preds == Y_test)/len(preds)

0.3602941176470588




---
# STOP

In [ ]:
# I'm opening the csv of lyrics from songlyrics.com, which is structured differently than the genius one.
# might need to restructure this code (or the genius csv) if we want to use that too
# the only difference is all the lyrics by each artist are combined into a single cell (as opposed to split by song)
df = pd.read_csv('data/all_sl.csv')
print(f'Imported {len(df)} lines')
genres = set(df['Genre'])
print(f'Identified {len(genres)} genres: {genres}')
lyrics = defaultdict(str)
for genre in genres:
    for artist in list(df[df['Genre'] == genre]['Lyrics']): # grab each row (artist) and select the lyrics cell
        lyrics[genre] += artist # combine each batch of lyrics into the respective genre in the dictionary
print(f'Extracted {len(lyrics)} genres')

Imported 166 lines
Identified 7 genres: {'soul_rb', 'alternative', 'rock', 'metal', 'rap_hiphop', 'pop', 'country'}
Extracted 7 genres


In [ ]:
frequency_tables = {}
words_uniq = {}
lines_uniq = {}
for (genre,lyric) in lyrics.items():
    frequency_tables[genre] = sl.words_freq(lyric, stopwords)
    words_uniq[genre] = sl.words(lyric, stopwords)
    print(f'[{genre}] Identified {len(words_uniq[genre])} unique words')

[pop] Identified 7269 unique words
[rap_hiphop] Identified 15104 unique words
[rock] Identified 6416 unique words
[metal] Identified 8963 unique words
[alternative] Identified 5750 unique words
[country] Identified 6533 unique words
[soul_rb] Identified 4626 unique words


In [ ]:
len(lyrics['alternative'])

703734

In [ ]:
# as an example, print the top 25 most common terms from each genre!
for genre in frequency_tables:
    print(f'### {genre} ###\n{sorted(frequency_tables[genre].items(), key=lambda x: x[1], reverse=True)[:25]}')

### pop ###
[('love', 2008), ('know', 1645), ('like', 1463), ('baby', 1121), ('got', 1087), ('yeah', 1053), ('go', 1007), ('get', 967), ('one', 868), ('cause', 867), ('let', 806), ('girl', 796), ('say', 727), ('make', 723), ('la', 709), ('want', 688), ('wanna', 667), ('time', 656), ('never', 643), ('need', 611), ('take', 606), ('right', 605), ('way', 580), ('heart', 573), ('see', 566)]
### rap_hiphop ###
[('like', 2874), ('got', 2079), ('get', 1991), ('nigga', 1790), ('know', 1750), ('aint', 1316), ('yeah', 1286), ('niggas', 1236), ('bitch', 1206), ('shit', 1184), ('fuck', 1143), ('love', 943), ('back', 891), ('see', 860), ('money', 799), ('go', 784), ('cause', 770), ('baby', 708), ('thats', 705), ('want', 702), ('man', 674), ('make', 669), ('em', 668), ('say', 660), ('right', 591)]
### rock ###
[('love', 1159), ('yeah', 869), ('know', 793), ('like', 679), ('got', 659), ('one', 628), ('away', 580), ('go', 551), ('get', 525), ('never', 525), ('time', 524), ('want', 520), ('come', 440), 

In [ ]:
# make a 80/20 train/test split of lyric words tagged by origin genre
X = []
Y = []
for (genre,words) in words_uniq.items():
    X += words
    Y += [genre] * len(words)
X_train, X_test, Y_train, Y_test = sk.model_selection.train_test_split(X, Y, test_size=0.2)
print(f'Split size: {len(X_test) / (len(X_test) + len(X_train))}\nTest set: {len(X_test)}\nTrain set: {len(X_train)}')


NameError: ignored